In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, AdaBoostClassifier, BaggingClassifier
from sklearn.cross_validation import StratifiedShuffleSplit
from sklearn.metrics import log_loss
from sklearn.linear_model import LogisticRegression
import datetime

In [2]:
df_train = pd.read_csv('raw_data/train.csv')

In [3]:
def convert_class_label_to_int(class_label):
    return int(class_label[6:]) - 1

In [4]:
def find_means_stds(features_array):
    means = [np.mean(features_array[:, i]) for i in range(features_array.shape[1])]
    stds = [np.std(features_array[:, i]) for i in range(features_array.shape[1])]
    return {'means': means, 'stds': stds}

def z_score_feature(feature_slice, mean, std):
    return (feature_slice - mean) / std

In [5]:
renamed_labels = [convert_class_label_to_int(i) for i in df_train['target'].values]
feature_columns = ['feat_' + str(i + 1) for i in range(93)]
df_train['renamed_labels'] = renamed_labels

In [6]:
mstddict = find_means_stds(df_train[feature_columns].values)
for i in range(93):
    df_train['feat_' + str(i + 1)] = z_score_feature(df_train['feat_' + str(i + 1)].values, mstddict['means'][i],
                                                     mstddict['stds'][i])

In [9]:
scoress = []
sss = StratifiedShuffleSplit(df_train['renamed_labels'], n_iter=3, test_size=0.2, random_state=42)
folder_id = 1
curr_item = 0
write_files = True
# write_files = False
for train_index, test_index in sss:
#     clf2 = RandomForestClassifier(n_estimators=400, max_features=25, min_samples_split=3, verbose=1)
    clf = GradientBoostingClassifier(n_estimators=700, max_depth=7, max_features=20, learning_rate=0.03)
#     clf = BaggingClassifier(base_estimator=clf2, n_estimators=5)
#     clf2 = AdaBoostClassifier(base_estimator=clf3, n_estimators=15)
#     clf = CalibratedClassifierCV(base_estimator=clf2, method='isotonic')
#     clf = AdaBoostClassifier(base_estimator=clf2, n_estimators=20)
    data_train = df_train[feature_columns].values[train_index, :]
    data_test = df_train[feature_columns].values[test_index, :]
    labels_train = df_train['renamed_labels'].values[train_index]
    labels_test = df_train['renamed_labels'].values[test_index]
    clf.fit(data_train, labels_train)
    pred = clf.predict_proba(data_test)
    res = log_loss(labels_test, pred)
    print(res)
    scoress.append(res)
    
    if write_files:
        predict_dict = {'Class_1': pred[:, 0]}
        for j in range(8):
            predict_dict['Class_' + str(j + 2)] = pred[:, j+1]
        df_sub = pd.DataFrame(predict_dict)
        fname = 'tmpsubs/' + str(folder_id) + '/gb_lr05_12_' + str(curr_item+1) + '.csv'
        df_sub.to_csv(fname, index=False)
    curr_item += 1
print(scoress, sum(scoress) / len(scoress))

0.462545722068
0.474198263909
0.459274852433
[0.46254572206801192, 0.47419826390899533, 0.45927485243277466] 0.465339612803
